In [2]:

# Pre-processing dataframe by:
#      - Exluding trusts with fewer than 500 patients
#      - Homogenising date formats
#      - Keeping episodes corresponding to first occurence of Covid for each patient

def PreProcessing(df_ini):
    import pandas as pd
    
    df2 = df_ini
    
    # Homogenising format for EPIstart and EPIend
#     ColumnList = ['Admission_date', 'Discharge_date', 'EPIstart', 'EPIend']
#     for item in ColumnList:
#         Date_Filter = df2[item].apply(str).str.contains('\d{5}', regex=True)
#         DateToConvert =  df2[item][Date_Filter]
#         DateConverted =  pd.to_datetime( DateToConvert.astype(int), unit='D', origin='1899-12-30' ) 
#         itemUpdated = item+'_Updated'
#         df2[itemUpdated] = df2[item]
#         df2[itemUpdated][Date_Filter] = DateConverted
#     #     df2[itemUpdated].iloc[list(DateConverted.index)] = DateConverted
#         df2[itemUpdated] = pd.to_datetime(df2[itemUpdated])

#     # Sort by episode start date and keep first instance of U071/U072 codes
# #     df2 = df2.sort_values(by=['EPIstart_Updated'])
# #     df2 = df2.drop_duplicates(subset='P_Spell_ID', keep='first')

#     ColumnList = ['P_Spell_ID', 'EPIstart_Updated']
#     df_FirstEpisode = df2[ColumnList].groupby('P_Spell_ID')['EPIstart_Updated'].first()

#     df2 = df2.merge(df_FirstEpisode, on='P_Spell_ID')
#     df2 = df2.drop('EPIstart_Updated_x', axis=1)
#     df2 = df2.rename(columns={'EPIstart_Updated_y':'EPIstart_Updated'})
#     df2 = df2.drop_duplicates(subset='P_Spell_ID', keep='first')

    # Homogenising format for EPIstart and EPIend
    ColumnList = ['Admission_date', 'Discharge_date', 'EPIstart', 'EPIend']
    for item in ColumnList:
#         df2[item] = pd.to_datetime( df2[item], format='%d/%m/%Y %H:%M' )
        df2[item] = pd.to_datetime( df2[item], format='%Y/%m/%d' )

    ColumnList = ['P_Spell_ID', 'EPIstart', 'HESID', 'Admission_date']
    
# First() method doesn't work, replaced with following paragraph
#     df_FirstEpisode = df2[ColumnList].groupby('P_Spell_ID')['EPIstart'].first()
#     df2 = df2.merge(df_FirstEpisode, on='P_Spell_ID')
#     df2 = df2.drop('EPIstart_x', axis=1)
#     df2 = df2.rename(columns={'EPIstart_y':'EPIstart'})
#     df2 = df2.drop_duplicates(subset='P_Spell_ID', keep='first')
    
#     ColumnGroup = ['HESID', 'Admission_date']
    ColumnGroup = ['P_Spell_ID']
    IndexFirstEpisode = df2[ColumnList].groupby(by=ColumnGroup)['EPIstart'].idxmin().values
    df2 = df2.loc[IndexFirstEpisode]
        
#     Creating column with patient counts for each trust
#     df2['Patients_Count'] = df2.groupby('ProvCode')['HESID'].transform('count')

#     # Keeping trusts with more than N patients within period
#     Nb_Patients = 500
#     NbPatients_Filter = df2['Patients_Count']>=Nb_Patients
#     df2 = df2[NbPatients_Filter]    

# Following Keith's advice, remove the condition on minimum number of infections recorded

    # Creating Column with difference between EPIstart and Admission_date
    df2['Diff_EPIstart_Admission'] = (df2['EPIstart'] - df2['Admission_date']).dt.days
    
    return df2

In [ ]:

# Creates column for NI / Not NI using method 2

def NI_M2_Identification(df_ini, Ndays):
    import numpy as np
    import pandas as pd
    
    df2 = df_ini
    
#     df2['Diff_EPIstart_Admission'] = ( pd.to_datetime(df2['EPIstart'], format='%d/%m/%Y %H:%M') - pd.to_datetime(df2['Admission_date'], format='%d/%m/%Y %H:%M') ).dt.days
    Ndays_Filter = df2['Diff_EPIstart_Admission'] >= Ndays
    df2['NC_Method2_'+str(Ndays)] = np.where(Ndays_Filter, 1, 0)

    return df2
